# Notebook explanation

1. Import necessary libraries

2. Prepare the dataset

3. Create the baseline regression model

4. Drift analysis with importances: default importances estimation

5. Drift analysis with importances: custom importances



In [1]:
! pip install evidently

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.5/555.5 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.

In [6]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

# 1. Import necessary libraries

In [7]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

from datetime import datetime, time
from sklearn import datasets, ensemble

from evidently import ColumnMapping
from evidently.test_suite import TestSuite
from evidently.report import Report
from evidently.metrics import DataDriftTable
from evidently.tests import TestShareOfDriftedColumns

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# 2. Prepare the dataset

In [8]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip", verify=False).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

In [9]:
raw_data.head()

instant  season  yr  mnth  hr  holiday  weekday  workingday  \
dteday                                                                    
2011-01-01        1       1   0     1   0        0        6           0   
2011-01-01        2       1   0     1   1        0        6           0   
2011-01-01        3       1   0     1   2        0        6           0   
2011-01-01        4       1   0     1   3        0        6           0   
2011-01-01        5       1   0     1   4        0        6           0   

            weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
dteday                                                                          
2011-01-01           1  0.24  0.2879  0.81        0.0       3          13   16  
2011-01-01           1  0.22  0.2727  0.80        0.0       8          32   40  
2011-01-01           1  0.22  0.2727  0.80        0.0       5          27   32  
2011-01-01           1  0.24  0.2879  0.75        0.0       3          10   13  
2011-01-01           1  0.24  0.2879  0.75        0.0       0           1    1

In [10]:
raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)

In [11]:
raw_data.head()

instant  season  yr  mnth  hr  holiday  weekday  \
2011-01-01 00:00:00        1       1   0     1   0        0        6   
2011-01-01 01:00:00        2       1   0     1   1        0        6   
2011-01-01 02:00:00        3       1   0     1   2        0        6   
2011-01-01 03:00:00        4       1   0     1   3        0        6   
2011-01-01 04:00:00        5       1   0     1   4        0        6   

                     workingday  weathersit  temp   atemp   hum  windspeed  \
2011-01-01 00:00:00           0           1  0.24  0.2879  0.81        0.0   
2011-01-01 01:00:00           0           1  0.22  0.2727  0.80        0.0   
2011-01-01 02:00:00           0           1  0.22  0.2727  0.80        0.0   
2011-01-01 03:00:00           0           1  0.24  0.2879  0.75        0.0   
2011-01-01 04:00:00           0           1  0.24  0.2879  0.75        0.0   

                     casual  registered  cnt  
2011-01-01 00:00:00       3          13   16  
2011-01-01 01:00:00       8          32   40  
2011-01-01 02:00:00       5          27   32  
2011-01-01 03:00:00       3          10   13  
2011-01-01 04:00:00       0           1    1

In [12]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [13]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [14]:
reference.shape , current.shape

((618, 16), (719, 16))

# 3. Baseline regression model

In [15]:
regressor = ensemble.RandomForestRegressor(random_state = 42, n_estimators = 50)

In [16]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

RandomForestRegressor(n_estimators=50, random_state=42)

In [17]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
current_prediction = regressor.predict(current[numerical_features + categorical_features])

In [22]:
pd.DataFrame(ref_prediction).head()

0
0  24.60
1  36.20
2  27.38
3  12.76
4   4.74

In [23]:
pd.DataFrame(current_prediction).head()

0
0  19.12
1  16.22
2  12.66
3   6.24
4   3.62

In [24]:
reference['prediction'] = ref_prediction
current['prediction'] = current_prediction

In [25]:
reference.head()

instant  season  yr  mnth  hr  holiday  weekday  \
2011-01-01 00:00:00        1       1   0     1   0        0        6   
2011-01-01 01:00:00        2       1   0     1   1        0        6   
2011-01-01 02:00:00        3       1   0     1   2        0        6   
2011-01-01 03:00:00        4       1   0     1   3        0        6   
2011-01-01 04:00:00        5       1   0     1   4        0        6   

                     workingday  weathersit  temp   atemp   hum  windspeed  \
2011-01-01 00:00:00           0           1  0.24  0.2879  0.81        0.0   
2011-01-01 01:00:00           0           1  0.22  0.2727  0.80        0.0   
2011-01-01 02:00:00           0           1  0.22  0.2727  0.80        0.0   
2011-01-01 03:00:00           0           1  0.24  0.2879  0.75        0.0   
2011-01-01 04:00:00           0           1  0.24  0.2879  0.75        0.0   

                     casual  registered  cnt  prediction  
2011-01-01 00:00:00       3          13   16       24.60  
2011-01-01 01:00:00       8          32   40       36.20  
2011-01-01 02:00:00       5          27   32       27.38  
2011-01-01 03:00:00       3          10   13       12.76  
2011-01-01 04:00:00       0           1    1        4.74

In [26]:
regressor.feature_importances_

array([3.02693536e-02, 3.31193604e-02, 5.38409022e-02, 2.13157227e-02,
       7.18431621e-01, 1.38268605e-02, 0.00000000e+00, 6.10781220e-04,
       1.28585399e-01])

# 4. Drift analysis with importances: default importances estimation

In [27]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [28]:
report = Report(metrics = [
    DataDriftTable(feature_importance=True)
])
report.run(reference_data=reference,
           current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
           column_mapping=column_mapping
           )

In [29]:
report.show()

In [30]:
report.save_html('report.html')

# 5. Drift analysis with importances: custom importances

In [31]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [32]:
report = Report(metrics = [
    DataDriftTable(feature_importance=True)
])
report.run(reference_data=reference,
           current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
           column_mapping=column_mapping,
           additional_data = {'current_feature_importance':
              dict(map(lambda i,j : (i,j), numerical_features + categorical_features, regressor.feature_importances_))
            }
           )

In [34]:
report.show()

In [35]:
report.save_html('report_v1.html')